In [2]:
!pip install owlready2 rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 63.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 31.9 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.47-cp311-cp311-linux_x86_64.whl size=24577500 sha256=78c5c330d537632710c0234e6c6ba289397bd2171daeaf2810492087e353f8a4
  Stored in directory: /root/.cache/pip/wheels/25/9a/a3/fb1ac6339caa859c8bb18d685736168b0b51d851af13d81d52
Successfully built owlready2


In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
from owlready2 import *

# Carregar CSV
df = pd.read_csv("equipamentos_manutencao.csv")

# Criar ontologia
onto = get_ontology("http://www.exemplo.org/manutencao_preditiva.owl")

with onto:

    # === CLASSES ===
    class Equipamento(Thing): pass
    class Sensor(Thing): pass
    class Componente(Thing): pass
    class Falha(Thing): pass
    class Manutencao(Thing): pass
    class Procedimento(Thing): pass
    class Responsavel(Thing): pass
    class Alerta(Thing): pass

    # === PROPRIEDADES DE OBJETO ===
    class possuiComponente(Equipamento >> Componente): pass
    class mede(Sensor >> Componente): pass
    class detectaFalha(Sensor >> Falha): pass
    class generaAlerta(Sensor >> Alerta): pass
    class executaProcedimento(Responsavel >> Procedimento): pass
    class associadoA(Manutencao >> Equipamento): pass

    # === PROPRIEDADES DE DADOS ===
    class frequenciaDeLeitura(DataProperty):
        domain = [Sensor]
        range = [float]

    class dataFalha(DataProperty):
        domain = [Falha]
        range = [str]

    class descricao(DataProperty):
        domain = [Falha]
        range = [str]

    class nome(DataProperty):
        domain = [Thing]
        range = [str]

# === POPULAR A ONTOLOGIA COM DADOS DO CSV ===
for _, row in df.iterrows():
    # Criar instâncias
    eqp = Equipamento(row['equipamento_id'])
    eqp.nome = [row['equipamento_nome']]

    cmp = Componente(row['componente_id'])
    cmp.nome = [row['componente_nome']]
    eqp.possuiComponente.append(cmp)

    sns = Sensor(row['sensor_id'])
    sns.nome = [row['sensor_nome']]
    sns.frequenciaDeLeitura = [row['frequencia_leitura']]
    sns.mede.append(cmp)

    flh = Falha(row['falha_id'])
    flh.nome = [row['falha_nome']]
    flh.dataFalha = [row['data_falha']]
    flh.descricao = [row['descricao_falha']]
    sns.detectaFalha.append(flh)

    alerta = Alerta(f"alerta_{row['sensor_id']}")
    sns.generaAlerta.append(alerta)

    tec = Responsavel(row['tecnico_id'])
    tec.nome = [row['tecnico_nome']]

    prc = Procedimento(row['procedimento_id'])
    prc.nome = [row['procedimento_nome']]
    tec.executaProcedimento.append(prc)

    manut = Manutencao(f"manut_{row['equipamento_id']}")
    manut.associadoA.append(eqp)

# === SALVAR A ONTOLOGIA ===
onto.save(file="ontologia_manutencao_populada.owl", format="rdfxml")
print("Ontologia gerada e salva como ontologia_manutencao_populada.owl")


Ontologia gerada e salva como ontologia_manutencao_populada.owl


In [8]:
# -*- coding: utf-8 -*-
from owlready2 import *

# Carregar ontologia populada
onto = get_ontology("ontologia_manutencao_populada.owl").load()


# Exemplo 1 – Listar todos os sensores e suas frequências de leitura
# print("\nSensores e frequência de leitura:")
#for s in onto.Sensor.instances():
#    print(f"{s.name}: {s.frequenciaDeLeitura[0]} Hz")


# Exemplo 2 – Mostrar falhas detectadas por um sensor específico
sensor_escolhido = "sns493"
sensor = onto.search_one(iri="*" + sensor_escolhido)
if sensor:
    print(f"\nFalhas detectadas por {sensor.nome[0]}:")
    for f in sensor.detectaFalha:
        print(f"- {f.nome[0]} em {f.dataFalha[0]}: {f.descricao[0]}")


# Exemplo 3 – Mostrar componentes que cada equipamento possui
#print("\nEquipamentos e seus componentes:")
#for e in onto.Equipamento.instances():
#    print(f"{e.nome[0]} possui:")
#    for c in e.possuiComponente:
#        print(f"  - {c.nome[0]}")



Falhas detectadas por Sensor de Vibração:
- Falha por Vibração Excedente em 2024-11-19: Perda de pressão no componente


# Exerícios

1. Quantos sensores existem na ontologia?

1. Qual é a frequência de leitura mais comum entre os sensores?

1. Liste todos os tipos de falhas registradas.

1. Quais sensores detectaram falhas no ano de 2024? (Filtrar Falha.dataFalha por startswith("2024"))

1. Qual técnico executou o maior número de procedimentos diferentes? (Contar as instâncias de Responsavel e executaProcedimento)